In [2]:
import numpy as np
import visa
import time

Mag_IStep=0.06# at least0.02, otherwise the instrument can get stuck. 0.10238 T/A, max 5.0 T=48.8A, 0.032 T/s=0.3A/s
Mag_IDelay=0.4
Mag_Resist=0.01# to be measured first!!!!!
Mag_VStep=0.005 #Mag_Resist*Mag_IStep
MagI_max=47
Mag_tot_V_max=0.7
test_curr =1
Meas_Resist_delay = 4
magnet_stable_delay = 5
# initialize 6031.
def B_to_IMag(B):
    return B/0.10238
def IMag_to_B(IMag):
    return IMag*0.10238


def initMag_U(instr):
    rm = visa.ResourceManager()
    hp6031 = rm.open_resource(instr)
    # set various k2400 parameters; see k2400 manual for the full selection
    if one_machine:
        hp6031.write('ISET 0') # set voltage to zero
        #hp6031.write('VSET %1.6f'%Mag_tot_V_max) # set voltage to zero
        
        vstep = Mag_VStep
        istep = Mag_VStep/Mag_Resist
        delay = istep/0.3/10# factor of 10 faster than as if ramping the current
        #cur_i = IreadMag(hp6031)# the upper is more accurate
        cur_v = VreadSetMag(hp6031)
        #print(cur_i)
        while abs(cur_v-Mag_tot_V_max) > vstep:
            #print(abs(cur_v-Mag_tot_V_max))
            if cur_v < Mag_tot_V_max:
                VsetMag(hp6031,cur_v+vstep)
                cur_v = cur_v+vstep
            else:
                VsetMag(hp6031,cur_v-vstep)
                cur_v = cur_v-vstep
            time.sleep(delay)
            
        VsetMag(hp6031,Mag_tot_V_max)
        
    else:
        hp6031.write('VSET 0') # set voltage to zero
        hp6031.write('ISET 47') # set voltage to zero        
    hp6031.write('IMAX 47A') # set current max
    #k2400.write('FORM:ELEM VOLT, CURR') # set communications formatting 
    #k2400.write('OUTP ON') # turn on output
    #k2400.write('CURR:PROT 100e-3') # set current compliance
    return hp6031


def initMag_D(instr):
    rm = visa.ResourceManager()
    hp6031 = rm.open_resource(instr)
    # set various k2400 parameters; see k2400 manual for the full selection
    hp6031.write('VSET 0') # set voltage to zero
    hp6031.write('ISET 47') # set voltage to zero
    hp6031.write('IMAX 47A') # set current max
    #k2400.write('FORM:ELEM VOLT, CURR') # set communications formatting 
    #k2400.write('OUTP ON') # turn on output
    #k2400.write('CURR:PROT 100e-3') # set current compliance
    return hp6031

# read I, V
def VreadMag(hp6031):
    return float(hp6031.query_ascii_values('VOUT?',converter='s')[0][6:11])
def VreadSetMag(hp6031):
    return float(hp6031.query_ascii_values('VSET?',converter='s')[0][6:11])
        
def IreadMag(hp6031):
    return float(hp6031.query_ascii_values('IOUT?',converter='s')[0][6:11])
def IreadSetMag(hp6031):
    return float(hp6031.query_ascii_values('ISET?',converter='s')[0][6:11])



# set dc current
def IsetMag(hp6031,target_i):
    return hp6031.write('ISET %1.6f'%target_i)
# Just in case we need to set V
def VsetMag(hp6031,target_v):
    return hp6031.write('VSET %1.6f'%target_v)



# sweep the current to target_i in small  steps with a specified delay.
# differs from IsetMag, which sets current as fast as the instrument will go.
def IsweepMag(hp6031,target_i):
    if no_sweep_just_set:
        # for roughly sweeping slowly        
        #vstep = Mag_VStep
        istep = Mag_IStep
        delay = Mag_IDelay
        #cur_i = IreadMag(hp6031)# the upper is more accurate
        cur_i = IreadSetMag(hp6031)
        #print(cur_i)
        while abs(cur_i-target_i) > istep:
            #print(abs(cur_v-Mag_tot_V_max))
            if cur_i < target_i:
                IsetMag(hp6031,cur_i+istep)
                cur_i = cur_i+istep
            else:
                IsetMag(hp6031,cur_i-istep)
                cur_i = cur_i-istep

            time.sleep(delay)
            
        IsetMag(hp6031,target_i)

        iread=IreadMag(hp6031)
        time.sleep(magnet_stable_delay)
        while(IreadMag(hp6031)!=iread):
            iread=IreadMag(hp6031)
            time.sleep(magnet_stable_delay)

    elif sweep_by_V:
        IsetMag(hp6031,MagI_max)

        vstep = Mag_VStep
        istep = Mag_VStep/Mag_Resist
        delay = istep/0.2
        cur_i = IreadMag(hp6031)# the upper is more accurate
        cur_v = VreadMag(hp6031)
        #print(cur_i)
        while abs(cur_i-target_i) > istep:
            print(abs(cur_i-target_i))
            if cur_i < target_i:
                VsetMag(hp6031,cur_v+vstep)
            else:
                VsetMag(hp6031,cur_v-vstep)

            time.sleep(delay)
            cur_i = IreadMag(hp6031)
            cur_v = VreadMag(hp6031)

        VsetMag(hp6031,cur_v+(target_i-cur_i)*Mag_Resist)
    else:
        #VsetMag(hp6031,Mag_tot_V_max)
        step=Mag_IStep
        delay=Mag_IDelay
        cur_i = IreadMag(hp6031)
        print(cur_i)
        while abs(cur_i-target_i) > step:
            print(abs(cur_i-target_i))
            if cur_i < target_i:
                IsetMag(hp6031,cur_i+step)
            else:
                IsetMag(hp6031,cur_i-step)
            time.sleep(delay)
            cur_i = IreadMag(hp6031)
        IsetMag(hp6031,target_i)    
    
def MeasMagR(hp6031_U,hp6031_D):## SHOULD only apply this at every thing initialized. 
    # let D do nothing
    global Mag_Resist,Mag_VStep
    delay=Mag_IDelay
    VsetMag(hp6031_D,0)
    IsetMag(hp6031_D,MagI_max)
    #let U show
    IsweepMag(hp6031_U,0)
    time.sleep(Meas_Resist_delay)
    VsetMag(hp6031_U,Mag_tot_V_max/2)
    time.sleep(Meas_Resist_delay)
    IsweepMag(hp6031_U,test_curr)
    time.sleep(Meas_Resist_delay)
    Mag_Resist=VreadMag(hp6031_U)/IreadMag(hp6031_U)
    Mag_VStep = Mag_Resist*Mag_IStep
    # change back to controling V
    IsweepMag(hp6031_U,0)
    if one_machine == 0:
        VsetMag(hp6031_U,0)
        IsetMag(hp6031_U,MagI_max)


    return Mag_Resist
    

# in the case of series, should use constant voltage mode, and I to maximum current for magnet
def IsweepBoth(hp6031_U,hp6031_D,target_i):
    IsetMag(hp6031_U,MagI_max)
    IsetMag(hp6031_D,MagI_max)

    vstep = Mag_VStep
    istep = Mag_IStep
    delay = Mag_IDelay
    cur_i = IreadMag(hp6031_U)# the upper is more accurate
    cur_v = VreadMag(hp6031_D)+VreadMag(hp6031_U)
    #print(cur_i)
    while abs(cur_i-target_i) > istep:
        print(abs(cur_i-target_i))
        if cur_i < target_i:
            VsetMag(hp6031_U,cur_v/2+vstep/2)
            VsetMag(hp6031_D,cur_v/2+vstep/2)
        else:
            VsetMag(hp6031_U,cur_v/2-vstep/2)
            VsetMag(hp6031_D,cur_v/2-vstep/2)

        time.sleep(delay)
        cur_i = IreadMag(hp6031_U)
        cur_v = VreadMag(hp6031_D)+VreadMag(hp6031_U)
        
    VsetMag(hp6031_U,cur_v/2+(target_i-cur_i)*Mag_Resist/2)
    VsetMag(hp6031_D,cur_v/2+(target_i-cur_i)*Mag_Resist/2)
